In [52]:
import numpy as np
import pandas as pd
from shutil import copyfile
from astropy.cosmology import Planck13 as cosmo
import fsps
from matplotlib import gridspec
from astropy.io import fits
from astropy import wcs
from astropy.modeling.models import Sersic2D
from astropy.table import Table
from sim_engine import Scale_model
from spec_tools import Source_present, Oldest_galaxy, Sig_int, Smooth
from spec_stats import Smooth, Highest_density_region
from spec_id import *
from spec_stats import Highest_density_region, Linear_fit
from spec_exam import Gen_spec
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from glob import glob
import seaborn as sea
import os
from grizli import multifit
from grizli import model
from sim_engine import forward_model_grism
from astropy.wcs import wcs

import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
R = robjects.r
pandas2ri.activate()

from time import time
sea.set(style='white')
sea.set(style='ticks')
sea.set_style({'xtick.direct'
               'ion': 'in','xtick.top':True,'xtick.minor.visible': True,
               'ytick.direction': "in",'ytick.right': True,'ytick.minor.visible': True})
cmap = sea.cubehelix_palette(12, start=2, rot=.2, dark=0, light=1.0, as_cmap=True)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

### set home for files
hpath = os.environ['HOME'] + '/'

if hpath == '/Users/Vince.ec/':
    dpath = '/Volumes/Vince_research/Data/' 
    
else:
    dpath = hpath + 'Data/' 

In [59]:
buldge_cat = Table.read(fits.open('../dataframes/galaxy_frames/goods_bulge.fits'), format = 'fits').to_pandas()

In [60]:
SF_GSD = pd.read_pickle('../Casey_data/massMetal_GSD.pkl')
SF_GND = pd.read_pickle('../Casey_data/massMetal_GND.pkl')

Q_GSD = pd.read_pickle('../dataframes/galaxy_frames/GS_cnd.pkl')
Q_GND = pd.read_pickle('../dataframes/galaxy_frames/GN_cnd.pkl')

In [61]:
def find_cand_id(gal_ra, gal_dec, cat_ra, cat_dec, cat_id, galid):
    r = (1. / 60 / 60) * 0.05
    for i in range(len(gal_ra)):
        cosr = np.sin(np.radians(gal_dec[i])) * np.sin(np.radians(cat_dec)) + np.cos(np.radians(gal_dec[i]))\
                * np.cos(np.radians(cat_dec)) * np.cos(np.radians(gal_ra[i]) - np.radians(cat_ra))
        rad = np.arccos(cosr)
        rad = np.degrees(rad)
        for ii in range(len(rad)):
            if rad[ii] < r:
                galid[i] = cat_id[ii]
    return galid

In [62]:
galid = np.repeat(-99.0, len(buldge_cat.ra))

inouts = find_cand_id(buldge_cat.ra.values,  buldge_cat.dec.values, Q_GSD.ra.values, Q_GSD.dec.values, Q_GSD.id.values, galid)
inouts = find_cand_id(buldge_cat.ra.values,  buldge_cat.dec.values, Q_GND.ra.values, Q_GND.dec.values, Q_GND.id.values, galid)
inouts = find_cand_id(buldge_cat.ra.values,  buldge_cat.dec.values, SF_GSD.RA.values, SF_GSD.DEC.values, SF_GSD.ID.values, galid)
inouts = find_cand_id(buldge_cat.ra.values,  buldge_cat.dec.values, SF_GND.RA.values, SF_GND.DEC.values, SF_GND.ID.values, galid)


In [64]:
buldge_cat['threeDHST_id'] = np.array(inouts).astype(int)

In [66]:
len(buldge_cat.query('threeDHST_id > 0'))

76

In [69]:
incat = buldge_cat.query('threeDHST_id > 0')

In [110]:
mdb = pd.read_pickle('../dataframes/fitdb/fullfitdb.pkl')
SFdb = pd.read_pickle('../Casey_data/SF_db_p1.pkl')

In [102]:
buldge_cat

,idx,candels_id,ra,dec,field,Hmag,UV_col,z,z_type,z_phot,...,stack_flux_100um,stack_flux_160um,stack_cov_100um,stack_cov_160um,flux_100um,flux_160um,flux_250um,flux_350um,flux_500um,threeDHST_id
0,236,237,53.108734,-27.941780,1,20.090000,1.800001,0.5197,1,0.531,...,0.190091,0.219145,177.484205,349.054038,-99.00000,-99.00000,-99.000000,-99.000000,-99.000000,-99
1,305,306,53.089714,-27.938513,1,21.930000,1.900000,1.4830,0,1.483,...,0.493474,2.691415,197.710665,389.888223,-99.00000,-99.00000,-99.000000,-99.000000,-99.000000,-99
2,363,364,53.105831,-27.936079,1,22.459999,1.799999,1.2520,0,1.252,...,-0.225823,-0.826958,195.031623,386.778483,-99.00000,-99.00000,-99.000000,-99.000000,-99.000000,-99
3,481,482,53.103519,-27.933323,1,21.990000,1.540001,1.0490,1,1.109,...,0.157994,-0.542957,207.549493,411.343797,-99.00000,-99.00000,-99.000000,-99.000000,-99.000000,-99
4,502,503,53.081944,-27.933586,1,19.790001,2.100000,0.6749,1,0.674,...,-0.255356,-0.816539,213.040261,421.794356,-99.00000,-99.00000,-99.000000,-99.000000,-99.000000,-99
5,513,514,53.131786,-27.934723,1,18.620001,2.070000,0.3970,0,0.397,...,0.522051,-0.066666,186.548228,369.895331,-99.00000,-99.00000,-99.000000,-99.000000,-99.000000,-99
6,527,528,53.113468,-27.933294,1,19.889999,1.610001,1.0887,1,0.997,...,-99.000000,-99.000000,201.827978,400.105925,6.26616,10.66880,12.065614,-99.000000,-99.000000,-99
7,579,580,53.087322,-27.929548,1,22.530001,1.719999,2.3530,0,2.625,...,-0.079515,0.713178,228.733623,435.365454,-99.00000,-99.00000,2.958808,-99.000000,-99.000000,-99
8,803,804,53.137962,-27.923832,1,20.299999,1.960001,0.6720,1,0.638,...,0.016922,5.378526,214.509344,424.633894,-99.00000,-99.00000,-99.000000,-99.000000,-99.000000,-99
9,871,872,53.116894,-27.924069,1,19.270000,2.029999,0.6650,1,0.626,...,0.031536,0.099166,232.905101,452.371092,-99.00000,-99.00000,-99.000000,-99.000000,-99.000000,-99


In [111]:
inout = np.repeat(0, len(mdb))
idx = 0
for i in mdb.index:
    if mdb.field[i] == 'GSD' and mdb.id[i] in incat.query('field == 1').threeDHST_id.values:
        inout[idx] = incat.query('field == 1 and threeDHST_id == {}'.format(mdb.id[i])).candels_id.values[0]
        
    if mdb.field[i] == 'GND' and mdb.id[i] in incat.query('field == 2').threeDHST_id.values:
        inout[idx] = incat.query('field == 2 and threeDHST_id == {}'.format(mdb.id[i])).candels_id.values[0]
        
    idx +=1
mdb['candels_id'] = inout
    
inout = np.repeat(0, len(SFdb))
idx = 0
for i in SFdb.index:
    if SFdb.field[i] == 'GSD' and SFdb.id[i] in incat.query('field == 1').threeDHST_id.values:
        inout[idx] = incat.query('field == 1 and threeDHST_id == {}'.format(SFdb.id[i])).candels_id.values[0]
        
    if SFdb.field[i] == 'GND' and SFdb.id[i] in incat.query('field == 2').threeDHST_id.values:
        inout[idx] = incat.query('field == 2 and threeDHST_id == {}'.format(SFdb.id[i])).candels_id.values[0]
        
    idx +=1
SFdb['candels_id'] = inout

print(idx)
# # print(incat.query('field == 1 and threeDHST_id > 40000').threeDHST_id)


69


In [117]:
Q_buldge = mdb.query('candels_id > 0')[['field','id','candels_id','lwa', 'lwa_hci', 'lwa_modality', 'Av','Av_hci', 'Av_modality', 'Z', 'Z_hci', 'Z_modality', 'lmass', \
                             'lmass_hci', 'lmass_modality', 'zgrism', 'zgrism_hci', 'zgrism_modality', 'z_50', 'z_50_hci', 'z_50_modality', 't_50', \
                             't_50_hci', 't_50_modality', 'log_ssfr', 'log_ssfr_hci', 'log_ssfr_modality']]

In [118]:
SF_buldge = SFdb.query('candels_id > 0')[['field','id','candels_id','lwa', 'lwa_hci', 'lwa_modality', 'Av','Av_hci', 'Av_modality', 'Z', 'Z_hci', 'Z_modality', 'lmass', \
                             'lmass_hci', 'lmass_modality', 'zgrism', 'z_50', 'z_50_hci', 'z_50_modality', 't_50', \
                             't_50_hci', 't_50_modality', 'log_ssfr', 'log_ssfr_hci', 'log_ssfr_modality']]

In [124]:
buldge_meas = pd.concat([Q_buldge, SF_buldge])

/Users/vestrada/miniconda3/envs/astroconda/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [127]:
buldge_meas = buldge_meas[['field','id','candels_id','lwa', 'lwa_hci', 'lwa_modality', 'Av','Av_hci', 'Av_modality', 'Z', 'Z_hci', 'Z_modality', 'lmass', \
                             'lmass_hci', 'lmass_modality', 'zgrism', 'z_50', 'z_50_hci', 'z_50_modality', 't_50', \
                             't_50_hci', 't_50_modality', 'log_ssfr', 'log_ssfr_hci', 'log_ssfr_modality']]

In [129]:
buldge_meas.to_csv('../dataframes/galaxy_frames/goods_bulge_stellar_params.dat',sep = ' ', index=False)
buldge_meas.to_pickle('../dataframes/galaxy_frames/goods_bulge_stellar_params.pkl')